In [21]:
import lightning as L
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import stats
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from objetos_otimizacao_rede_neural import *
from lightning.pytorch.loggers import TensorBoardLogger
from optuna import create_study, Trial
import os

In [22]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 0
NUM_EPOCAS = 2

In [23]:
logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs/")
treinador = L.Trainer(logger=False,enable_checkpointing=True,max_epochs=NUM_EPOCAS)


def build_model(trial):
    num_dados_de_entrada = 5
    num_dados_de_saida = 1
    neuronios_c1 = trial.suggest_int("c1_neuronios",1,30)
    neuronios_c2 = trial.suggest_int("c2_neuronios",1,30)
    neuronios_c3 = trial.suggest_int("c3_neuronios",1,30)


    minha_mlp = MLP(
        num_dados_de_entrada, neuronios_c1, neuronios_c2,neuronios_c3, num_dados_de_saida
    )
    
    return minha_mlp

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [24]:
def objective(trial):

    k = []
    for i in range(int(1/TAMANHO_TESTE)):
        print(i)
        dm = DataModule(TAMANHO_TESTE, SEMENTE_ALEATORIA,i)
        minha_mlp = build_model(trial)
        treinador.fit(minha_mlp, dm)

        minha_mlp.eval()
        dm.setup("test")

        with torch.no_grad():
            X_true = dm.X_teste

            y_true = dm.y_teste
            y_true = dm.y_scaler.inverse_transform(y_true)

            y_pred = minha_mlp(X_true)
            y_pred = dm.y_scaler.inverse_transform(y_pred)

            RMSE = root_mean_squared_error(y_true, y_pred)
        
        k.append(RMSE)

    rmse_medio = (sum(np.array(k)**2)/int(1/TAMANHO_TESTE))**.5
    return rmse_medio



In [25]:
study = create_study(direction='minimize')
parametros_totais = []

for _ in range(2):

    study.optimize(objective, n_trials=1)
    study.trials_dataframe().to_excel('triagem.xlsx')

[I 2024-05-01 20:39:18,703] A new study created in memory with name: no-name-f0696542-cf77-4e82-ade8-6f047e485961
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\trainer\configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory c:\Users\jose23038\OneDrive - ILUM ESCOLA DE CIÊNCIA\Semestre 3 - 2024.1\Redes-neurais-e-Algoritmos-Geneticos\Rede Neurais\Tarefas\Redes-Neurais\Otimização\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 915   
--------------------------------

0


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\trainer\configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory c:\Users\jose23038\OneDrive - ILUM ESCOLA DE CIÊNCIA\Semestre 3 - 2024.1\Redes-neurais-e-Algoritmos-Geneticos\Rede Neurais\Tarefas\Redes-Neurais\Otimização\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 915   
---------------------------------------
915       Trainable params
0         Non-trainable params
9

1
2
3
4
5


...

C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\trainer\configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory c:\Users\jose23038\OneDrive - ILUM ESCOLA DE CIÊNCIA\Semestre 3 - 2024.1\Redes-neurais-e-Algoritmos-Geneticos\Rede Neurais\Tarefas\Redes-Neurais\Otimização\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 339   
---------------------------------------
339       Trainable params
0         Non-trainable params
339       Total params
0.001     Total estimated

0
1
2
3
4
5


`Trainer.fit` stopped: `max_epochs=2` reached.
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\trainer\configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory c:\Users\jose23038\OneDrive - ILUM ESCOLA DE CIÊNCIA\Semestre 3 - 2024.1\Redes-neurais-e-Algoritmos-Geneticos\Rede Neurais\Tarefas\Redes-Neurais\Otimização\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 339   
---------------------------------------
339       Trainable params
0         Non-trainable params
3

6
7
8
9


[I 2024-05-01 20:39:31,163] Trial 1 finished with value: 797.9987147326889 and parameters: {'c1_neuronios': 23, 'c2_neuronios': 6, 'c3_neuronios': 7}. Best is trial 0 with value: 513.4663641264968.
